In [17]:
"""
Shows SpamAssassin Python Wrapper at work.

https://pypi.org/project/spamassassin-client/


- SETUP: -
1) Have spamasassin installed
    > sudo apt get spamassassin
2) > pip install spamassassin_client
3) Start spamassassin server
    > sudo spamd
4) Use spamassassin_client

- NOTE: -
REMEMBER TO SHUT DOWN SPAMD SERVER WHEN DONE

"""
import os
from spamassassin_client import SpamAssassin
import pandas as pd
import numpy as np

### Forming any text into valid email including headers and signatures
This is so we can ignore pesky SpamAssassin header and signature rules

In [18]:
# NOTE: Want to prepend this to any text to test that does not already have a header.
#           This will be the case for any text outside of the spam_assassin_corpus
#           This is from the sample-spam.txt file -> does not have any header issues
HAM_HEADER = """Subject: Example header
Message-ID: <GTUBE1.1010101@example.net>
Date: Wed, 23 Jul 2003 23:30:00 +0200
From: Sender <sender@example.net>
To: Recipient <recipient@example.net>
Precedence: junk
MIME-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
"""

header_lines = HAM_HEADER.split('\n')
REQ_HEADERS = {(line.split(':')[0]+':').lower() for line in header_lines if line != ''}
REQ_HEADERS

{'content-transfer-encoding:',
 'content-type:',
 'date:',
 'from:',
 'message-id:',
 'mime-version:',
 'precedence:',
 'subject:',
 'to:'}

In [19]:
def add_header(text):
    # NOTE: only tested on data/spam_ham_dataset.csv
    txt_lines = text.split('\n')
    header_lines = HAM_HEADER.split('\n')
    

In [20]:
spam_ham_df = pd.read_csv('data/spam_ham_dataset.csv')
spam = spam_ham_df[spam_ham_df['label'] == 'spam']
ham = spam_ham_df[spam_ham_df['label'] == 'ham']

spam_sample = spam.iloc[0]
ham_sample = ham.iloc[0]

# NOTE: looking at the format of the data.
#   We can see we have subject lines. Need to fit our header into there
# print("----SPAM----")
# print(spam_sample['text'])
# print("----HAM----")
# print(ham_sample['text'])

In [21]:
print(add_header(ham_sample['text']))

None
